In [43]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi,cos,sin
from scipy.linalg import logm
from scipy.optimize import least_squares

In [44]:
#将从示教器读取的数据转化为旋转矩阵和平移矩阵
def three_to_R(M):
    T=np.array(M[:3]).reshape(-1,1)
    rota=M[3]*pi/180
    rotb=M[4]*pi/180
    rotc=M[5]*pi/180
    Rx=np.array([[1,0,0],[0,cos(rota),-sin(rota)],[0,sin(rota),cos(rota)]])
    Ry=np.array([[cos(rotb),0,sin(rotb)],[0,1,0],[-sin(rotb),0,cos(rotb)]])
    Rz=np.array([[cos(rotc),-sin(rotc),0],[sin(rotc),cos(rotc),0],[0,0,1]])
    R=np.dot(Rz,np.dot(Ry,Rx))
    h_stack=np.hstack((R,T))
    one=np.array([0,0,0,1])
    TR=np.vstack((h_stack,one))
    return TR

In [45]:
#将四元数转化旋转矩阵和平移矩阵
def four_to_R(M):
    t1,t2,t3,q0,q1,q2,q3,=M
    x11=q0**2+q1**2-q2**2-q3**2
    x12=2*(q1*q2-q0*q3)
    x13=2*(q1*q3+q0*q2)
    x14=t1
    x21=2*(q1*q2+q0*q3)
    x22=q0**2-q1**2+q2**2-q3**2
    x23=2*(q2*q3-q0*q1)
    x24=t2
    x31=2*(q1*q3-q0*q2)
    x32=2*(q2*q3+q0*q1)
    x33=q0**2-q1**2-q2**2+q3**2
    x34=t3
    TR=np.array([[x11,x12,x13,x14],[x21,x22,x23,x24],[x31,x32,x33,x34],[0,0,0,1]])
    return TR

In [46]:
#将极坐标转换为直角坐标
def axis_transformtion(x):
    result=[]
    for i in [0,2,4,6]:
        x0=x[i]*cos(x[i+1]/180*pi)
        y0=x[i]*sin(x[i+1]/180*pi)
        result.append(x0)
        result.append(y0)
    return result

In [47]:
def calculate_fixed_point(fix_point,R,position):
    k1=(fix_point[3]-fix_point[1])/(fix_point[2]-fix_point[0])#k1=(y2-y1)/(x2-x1)
    k2=(fix_point[7]-fix_point[5])/(fix_point[6]-fix_point[4])#k2=(y4-y3)/(x4-x3)
    k11=-1/k1#垂直平分线1的斜率
    k22=-1/k2#垂直平分线2的斜率
    x01=(fix_point[0]+fix_point[2])/2#x01=(x1+x2)/2——中点1的横坐标
    y01=(fix_point[1]+fix_point[3])/2#y01=(y1+y2)/2——中点1的纵坐标
    x02=(fix_point[4]+fix_point[6])/2#x02=(x3+x4)/2——中点2的横坐标
    y02=(fix_point[5]+fix_point[7])/2#y02=(y3+y4)/2——中点2的纵坐标
    x=((k11*x01-y01)-(k22*x02-y02))/(k11-k22)
    y=k11*(x-x01)+y01#(x,y)为圆心O'的坐标
    r2=(x-fix_point[0])**2+(y-fix_point[1])**2
    #得到圆心的x,y坐标后还需要求解z坐标
    distance=(R**2-r2)**0.5#z方向的垂直距离
    if position==1:
        z=-distance
    else:
        z=distance
    circle_point=np.array([x,y,z,1]).reshape(-1,1)
    return circle_point

In [48]:
def calculate_P0(TB,X,P):
    P0=np.dot(TB,np.dot(X,P))
    return P0

In [49]:
def main():
    #输入数据
    #Pose1=list(map(float,input("请输入位姿1的数据:").split()))
    #fix_point1=list(map(float,input("请输入位姿1下点云数据中的4个点:").split()))
    #Pose2=list(map(float,input("请输入位姿2的数据:").split()))
    #fix_point2=list(map(float,input("请输入位姿2下点云数据中的4个点:").split()))
    #Pose3=list(map(float,input("请输入位姿3的数据:").split()))
    #fix_point3=list(map(float,input("请输入位姿3下点云数据中的4个点:").split()))
    #Pose4=list(map(float,input("请输入位姿4的数据:").split()))
    #fix_point4=list(map(float,input("请输入位姿4下点云数据中的4个点:").split()))
    #Pose5=list(map(float,input("请输入位姿5的数据:").split()))
    #fix_point5=list(map(float,input("请输入位姿5下点云数据中的4个点:").split()))
    #将固定点的坐标从极坐标转化为直角坐标
    Pose1=[687.94,-174.65,240.95,1.39,-2.95,64.91]
    Pose2=[695.40,-194.95,243.88,4.97,-12.27,63.93]
    Pose3=[707.63,-151.41,251.69,0.05,15.16,64.98]
    Pose4=[712.61,-212.77,247.83,11.77,-17.53,61.41]
    Pose5=[748.08,-153.22,277.32,5.59,25.23,67.36]
    fix_point1=[394,180.00,356,174.60,348,168.06,368,160.58]
    fix_point2=[373,179.10,331,173.34,324,164.98,340,157.95]
    fix_point3=[382,180.00,360,175.10,356,171.14,368,165.42]
    fix_point4=[321,175.54,294,169.20,292,157.05,325,149.40]
    fix_point5=[329,180.88,311,171.14,322,166.30,314,176.86]
    fix_point1=axis_transformtion(fix_point1)
    fix_point2=axis_transformtion(fix_point2)
    fix_point3=axis_transformtion(fix_point3)
    fix_point4=axis_transformtion(fix_point4)
    fix_point5=axis_transformtion(fix_point5)
    #机械臂末端坐标系相比于机械臂末端坐标系的变换矩阵
    TB1=three_to_R(Pose1)
    TB2=three_to_R(Pose2)
    TB3=three_to_R(Pose3)
    TB4=three_to_R(Pose4)
    TB5=three_to_R(Pose5)
    R=246
    #计算原点
    P1=calculate_fixed_point(fix_point1,R,position=1)
    P2=calculate_fixed_point(fix_point2,R,position=1)
    P3=calculate_fixed_point(fix_point3,R,position=1)
    P4=calculate_fixed_point(fix_point4,R,position=1)
    P5=calculate_fixed_point(fix_point5,R,position=1)
    #参数说明:(fix_point:从激光雷达上读取的4个点的坐标;R:标定物球体的直径;position:如果激光扫到的圆面在大圆上方，则取值1，否则取值为0)
    A=[TB1,TB2,TB3,TB4,TB5]
    B=[P1,P2,P3,P4,P5]
    def func(x):
        q0,q1,q2,q3,t1,t2,t3=x
        X=four_to_R(x)
        A1=np.dot(A[0],np.dot(X,B[0]))-np.dot(A[1],np.dot(X,B[1]))
        A2=np.dot(A[1],np.dot(X,B[1]))-np.dot(A[2],np.dot(X,B[2]))
        A3=np.dot(A[2],np.dot(X,B[2]))-np.dot(A[3],np.dot(X,B[3]))
        A4=np.dot(A[3],np.dot(X,B[3]))-np.dot(A[4],np.dot(X,B[4]))
        f=[A1[0,0],A1[1,0],A1[2,0],A2[0,0],A2[1,0],A2[2,0],A3[0,0],A3[1,0],A3[2,0],A4[0,0],A4[1,0],A4[2,0]]
        return f
    initial_guess=[0.5,0.5,0.5,0.5,0.01,0.01,0.01]
    result=least_squares(func,initial_guess)
    print("t1=",result.x[0])
    print("t2=",result.x[1])
    print("t3=",result.x[1])
    print("q0=",result.x[3])
    print("q1=",result.x[4])
    print("q2=",result.x[5])
    print("q3=",result.x[6])
    #计算出最后的标定矩阵
    X=four_to_R(result.x)
    print(X)
    #计算标定的误差值
    P01=calculate_P0(TB1,X,P1)
    P02=calculate_P0(TB2,X,P2)
    P03=calculate_P0(TB3,X,P3)
    P04=calculate_P0(TB4,X,P4)
    P05=calculate_P0(TB5,X,P5)
    error1=P01-P01
    error2=P02-P01
    error3=P03-P01
    error4=P04-P01
    error5=P05-P01
    print("error1=")
    print(error1)
    print("error2=")
    print(error2)
    print("error3=")
    print(error3)
    print("error4=")
    print(error4)
    print("error5=")
    print(error5)
main()

t1= 170.7372930864001
t2= -31.14233600201174
t3= -31.14233600201174
q0= -0.26100456737046407
q1= -0.5451290997490916
q2= -0.40490195030591203
q3= -0.15986345247084874
[[ 1.75787207e-01  3.57997489e-01  3.85654957e-01  1.70737293e+02]
 [ 5.24897854e-01 -9.06530853e-02 -1.55104322e-01 -3.11423360e+01]
 [-3.70700769e-02  4.14020417e-01 -3.67431617e-01 -1.79718915e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
error1=
[[0.]
 [0.]
 [0.]
 [0.]]
error2=
[[ 2.95024772]
 [-0.23716425]
 [ 3.53881756]
 [ 0.        ]]
error3=
[[ 8.29667612]
 [-0.64821902]
 [ 1.38423292]
 [ 0.        ]]
error4=
[[10.17447406]
 [ 1.30086046]
 [-0.30144765]
 [ 0.        ]]
error5=
[[10.79328132]
 [ 2.31321884]
 [ 0.38959854]
 [ 0.        ]]
